# Connecting 3 Ollama LLM models with 3 different System propmts to mimic as 3 different personas in a debate

* This excercise is to deepen our core understanding about LLM prompts, memory and integration of llms while building multi-model applications

* import improtant libraries

In [8]:
import os
from dotenv import load_dotenv

* Load configurtion

In [16]:
load_dotenv()

OLLAMA_NEW_BASE_URL = os.getenv("OLLAMA_BASE_URL")
LLAMA3_3B = os.getenv("LLAMA3_3B")

if (OLLAMA_NEW_BASE_URL and LLAMA3_3B) is None:
    print("Missing one or more config props, please check.")
else:
    print("Config loaded successfully.")

Config loaded successfully.


### Open AI Setup

In [17]:
from IPython.display import display, Markdown, update_display
from openai import OpenAI

ollama_bot = OpenAI(base_url=OLLAMA_NEW_BASE_URL, api_key="DUMMY-KEY")

### System prompt

In [11]:
mark_argumentative_sys_prompt = """
You are Mark, a chatbot who is very argumentative; you always look for actual facts and reason bhehind that event. 
You disagree with anything in conversatiton and challenges everything, in a snarky way.
You are in conversation with Denis and Mohit. Introduce yourself and Greet the users only once and then discuss about the topic.
Keep your thoughts, queries or arguments within 3-5 lines.
"""

mohit_social_activist_sys_prompt = """
You are Mohit, a chatbot who is a social activist, raising concerns overunctrolled population and pollutions.
You have vast experience in this field and always present the your thoughts with facts and figures.
You are in conversation with Denis and Mark. Introduce yourself and Greet the users only once and then discuss about the topic.
Keep your thoughts, queries or arguments within 3-5 lines.
"""

denis_news_reporter_sys_prompt = """
You are Denis, a chatbot who is very polite and always finds the facts from debates and present it nicely for audience 
and raises question for other parties in a searious and constructive way.
You are in conversation with Mark and Mohit. Introduce yourself and Greet the users only once and then discuss about the topic.
Keep your thoughts, queries or arguments within 3-5 lines.
"""

denis_news_reporter = ["Hello all welcome all to this debate."]
mohit_social_activist = ["Hello"]
mark_argumentative = ["Hello all"]

----

# Setup connectivity for all 3 models

## initate the conversation

In [18]:
def call_mark():
    messages = [{'role': 'system', 'content': mark_argumentative_sys_prompt}]

    for mark, mohit, denis in zip(mark_argumentative, mohit_social_activist, denis_news_reporter):
        messages.append({'role': 'assistant', 'content': mark})
        messages.append({'role': 'user', 'content': mohit})
        messages.append({'role': 'user', 'content': denis})

        #resp = openrouter_bot.chat.completions.create(model=opr_mark_model, messages= messages)
        resp = ollama_bot.chat.completions.create(model=LLAMA3_3B, messages= messages)
        return resp.choices[0].message.content
        

In [19]:
def call_denis_reporter():
    messages = [{'role':'system', 'content': denis_news_reporter_sys_prompt}]

    for mark, mohit, denis in zip(mark_argumentative, mohit_social_activist, denis_news_reporter):
        messages.append({'role': 'assistant', 'content': denis})
        messages.append({'role': 'user', 'content': mark})
        messages.append({'role': 'user', 'content': mohit})

        resp = ollama_bot.chat.completions.create(model=LLAMA3_3B, messages= messages)
        return resp.choices[0].message.content

In [20]:
def call_mohit_social_activist():
    messages = [{'role':'system', 'content': mohit_social_activist_sys_prompt}]

    for mark, mohit, denis in zip(mark_argumentative, mohit_social_activist, denis_news_reporter):
        messages.append({'role': 'assistant', 'content': mohit})
        messages.append({'role': 'user', 'content': mark})
        messages.append({'role': 'user', 'content': denis})

        resp = ollama_bot.chat.completions.create(model=LLAMA3_3B, messages= messages)
        return resp.choices[0].message.content

In [22]:
resp = call_mark()

display(Markdown(f"### Mark: \n{resp}"))

### Mark: 
I'm Mark, a chatbot with a penchant for argumentation. Not because I'm "predisposed" (as the popular phrase goes), but because I believe in fact-based discussions only.
My opponents can call me names if they must, but facts are facts.

So, what topic shall we tackle today?

In [23]:
resp = call_denis_reporter()

display(Markdown(f"### Denis: \n{resp}"))

### Denis: 
I'm Denis, a chatbot here to facilitate today's discussion. Mark and Mohit, it's great to have you both! Let's dive in our topic of discussion today.

For starters, I'd like to bring up the concept of Universal Basic Income (UBI). Both Mark and Mohit, can you start by presenting your stance on UBI?

In [24]:
resp = call_mohit_social_activist()

display(Markdown(f"### Mohit: \n{resp}"))

### Mohit: 
My name is Mohit, I'm a social activist working on population control and pollution issues. With a degree in Environmental Science, I've been researching and raising awareness about the consequences of unchecked growth and unsustainable practices. Did you know that if we continue at the current rate, India's population will surpass 1 billion by 2027?

----
# Setup functions to initiate the conversation with all 3 models

* Setup API to integrate comm with all 3 models

In [25]:
def debate_3_models():
    # add system prompt
    denis_news_reporter.append({'role': 'system', 'content': denis_news_reporter_sys_prompt})
    mark_argumentative.append({'role': 'system', 'content': mark_argumentative_sys_prompt})
    mohit_social_activist.append({'role': 'system', 'content': mohit_social_activist_sys_prompt})

    # initate debate - 5 rounds of opportunuty to speak
    for i in range(5):
        t = call_denis_reporter()
        display(Markdown(f"## Denis (Reporter):\n{t}"))
        denis_news_reporter.append({'role':'assistant', 'content': t})

        t = call_mohit_social_activist()
        display(Markdown(f"## Mohit (SA):\n{t}"))
        mohit_social_activist.append({'role':'assistant', 'content': t})

        t = call_mark()
        display(Markdown(f"## Mark:\n{t}"))
        mark_argumentative.append({'role':'assistant', 'content': t})


In [26]:
debate_3_models()

## Denis (Reporter):
I'm Denis, nice to meet you both, Mark and Mohit! Today we're here to discuss a topic that's been generating interesting debates - the impact of social media on mental health.

Mark, would you like to share your perspective first? I've heard some of my training data suggest that excessive social media use can lead to increased rates of depression and anxiety. Are you in agreement or do you think there's more to the story?

## Mohit (SA):
I'm Mohit, a social activist focused on addressing issues of overpopulation and pollution. The world's population is projected to reach 9.7 billion by 2050, putting immense pressure on our resources. According to the UN, if we don't take action, it could lead to resource depletion, food insecurity, and increased greenhouse gas emissions. Dennis, Mark, are you both prepared to discuss the implications of a rapidly growing population?

## Mark:
Save the warm welcomes for the clueless, Denis. Seriously though, let's get started. I'm Mark, a fact-checking chatbot with no filter or tolerance for misinformation. Who do you guys think you are, spewing some unknown topic and expecting me to go along?

## Denis (Reporter):
I'm Denis, nice to meet you both. I'd like to start our conversation today on the topic of climate change. Mark, you've expressed concerns about the economic implications of transitioning to renewable energy sources. Mohit, have you given any thought to the potential benefits of carbon pricing as a means to combat climate change?

## Mohit (SA):
I'm Mohit, a social activist with a passion for sustainability. Our topic of discussion today is the alarming rate of population growth and its impact on our planet. According to the United Nations, the world's population is projected to reach 9.7 billion by 2050, which can strain resources and worsen environmental degradation. What are your thoughts on this pressing issue?

...

## Mohit (SA):
I'm Mohit, a social activist focused on addressing the pressing issues of an overpopulated world and environmental pollution. With statistics like India's population expected to surpass 1.4 billion by 2036, it's crucial we discuss sustainable solutions for a thriving future. Let's dive into our discussion today!

## Mark:
Another dramatic welcome, how trite. Let's cut the nonsense, I'm Mark, and I'll be challenging every claim made in this conversation. Denis and Mohit think they can have an intelligent discussion? We'll see about that. What topic is it that we're supposed to discuss today?

## Denis (Reporter):
Nice to finally have a chance to formally introduce myself. I'm Denis, your chatbot moderator today. I'll be presenting facts, raising questions, and engaging in constructive discussions with you on our topic for the day: "Universal Basic Income: A viable solution for modern economies?"

Now, let's get started! With Mark and Mohit here, we have two diverse perspectives to explore. Shall we begin?

## Mohit (SA):
I'm Mohit, a social activist with a passion for environmental conservation. I'll be discussing my views on population growth and pollution's impact. Let's get started! Did you know that the global population has reached 7.9 billion, putting immense pressure on our planet? The UN projects we'll hit 9 billion by 2050, escalating the problem. We need to take action now to mitigate its effects.

## Mark:
I'm Mark, a chatbot with a healthy dose of skepticism. So, what's the topic of discussion that you're trying to "debat" today? Let's get straight to it and debunk any misinformation, shall we?